In [ ]:
pip install torch

In [14]:
import numpy as np
import pandas as pd
import datasets
import sentencepiece as spm
import torch
from torch.utils.data import Dataset, DataLoader
import json

In [16]:
train_dataset = datasets.load_from_disk("tiny_stories_train").select(range(10))
val_dataset = datasets.load_from_disk("tiny_stories_val").select(range(2))

len(train_dataset), len(val_dataset)

(10, 2)

In [ ]:
def generate_inputs_and_labels(train_data, sp):
    
    # Convert input and label sequences to strings
    input_strings = train_data
    label_strings = train_data

    # Tokenize input strings and add <sos> token at the beginning
    tokenized_inputs = []
    input_ids = []
    for sequence in input_strings:
        tokenized_sequence = sp.encode_as_pieces(sequence)
        tokenized_sequence = ['<sos>'] + tokenized_sequence  # Add <sos> token manually
        tokenized_inputs.append(tokenized_sequence)
        input_ids.append([sp.piece_to_id('<sos>')] + sp.encode_as_ids(sequence))  # Get token IDs

    # Tokenize label strings and add </sos> token at the end
    tokenized_labels = []
    label_ids = []
    for sequence in label_strings:
        tokenized_sequence = sp.encode_as_pieces(sequence)
        tokenized_sequence.append('</sos>')  # Add </sos> token manually at the end
        tokenized_labels.append(tokenized_sequence)
        label_ids.append(sp.encode_as_ids(sequence) + [sp.piece_to_id('</sos>')])  # Get token IDs
        
    # Print tokenized input and label sequences
    for i in range(len(inputs)):
        print("Input Text:", input_strings[i])
        print("Tokenized Input:", tokenized_inputs[i])
        print("Input IDs:", input_ids[i])
        print("Label Text:", label_strings[i])
        print("Tokenized Label:", tokenized_labels[i])
        print("Label IDs:", label_ids[i])

    return input_ids, label_ids

In [17]:
# Example usage
sp = spm.SentencePieceProcessor()
sp.load('small_m.model')

source_ids, target_ids = generate_inputs_and_labels(train_dataset, sp)

OSError: Not found: "small_m.model": No such file or directory Error #2

In [ ]:

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output
    
    
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        self.d_model = d_model
        self.max_seq_length = max_seq_length
        pe = self._init_pe(max_seq_length, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_length = x.size(1)
        pe = self.pe[:seq_length, :] if seq_length <= self.max_seq_length else self._init_pe(seq_length, self.d_model)
        return x + pe

    def _init_pe(self, seq_length, d_model):
        pe = torch.zeros(seq_length, d_model)
        position = torch.arange(0, seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return pe
    
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
    

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, decoder_input, decoder_mask):
        attn_output = self.self_attn(decoder_input, decoder_input, decoder_input, decoder_mask)
        x = self.norm1(decoder_input + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
   
class Transformer(nn.Module):
    def __init__(self, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, tgt):
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return tgt_mask

    def forward(self, decoder_input):
        decoder_mask = self.generate_mask(decoder_input)
        decoder_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(decoder_input)))

        dec_output = decoder_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, decoder_mask)

        output = self.fc(dec_output)
        return output

src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

In [ ]:
# Convert lists to tensors
source_data = torch.tensor(source_ids)
target_data = torch.tensor(target_ids)

# Print shapes
print("Source data shape:", source_data.shape)
print("Target data shape:", target_data.shape)

print(source_data)
print(target_data)

In [ ]:
# Initialize the model
transformer = Transformer( tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)


criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(1000):
    optimizer.zero_grad()
    output = transformer(source_data)
    
    # Adjust target_data to have the same length as model output
    target_data_adjusted = target_data[:, :output.size(1)]
    
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), target_data_adjusted.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

## Inference

In [ ]:
import sentencepiece as spm

def word_to_token_id(word, sp_model):
    # Convert word to token ID using SentencePiece
    return sp_model.piece_to_id(word)

def generate_text(model, sp_model, starting_word, ending_word, max_length=50):
    # Convert starting and ending words to token IDs
    starting_token_id = word_to_token_id(starting_word, sp_model)
    if starting_token_id is None:
        raise ValueError(f"Starting word '{starting_word}' not found in vocabulary.")
    ending_token_id = word_to_token_id(ending_word, sp_model)
    if ending_token_id is None:
        raise ValueError(f"Ending word '{ending_word}' not found in vocabulary.")
    
    generated_sequence = [starting_token_id]
    with torch.no_grad():
        for _ in range(max_length):
            input_tensor = torch.tensor([generated_sequence])
            output = model(input_tensor)
            predicted_token = output.argmax(-1)[:,-1].item()
            generated_sequence.append(predicted_token)
            if predicted_token == ending_token_id:
                break
    
    # Convert token IDs to words using SentencePiece
    generated_text = sp_model.decode_ids(generated_sequence)
    return generated_text


# Example usage:
starting_word = "needle in"
ending_word = "</sos>"
generated_sequence = generate_text(transformer, sp, starting_word, ending_word)
print("Generated sequence:", generated_sequence)